### 基于内容的推荐机制

在上个 notebook 中，我们介绍了如何使用协同过滤进行推荐。但是如果使用该技巧，有很多用户根本无法获得任何推荐电影。还有一些用户获得的推荐电影不到函数设置的十部电影上限。

为了向这些用户做出推荐，我们将尝试另一个技巧：**基于内容的**推荐机制。我们接着上个 notebook 继续。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

ModuleNotFoundError: No module named 'progressbar'

### 数据集

现在你可以访问将在这个 notebook 的后续部分一直使用的以下三大项了。  

`a.` **movies** - 一个 DataFrame，其中包含数据集中的所有电影，以及关于电影的其他内容相关信息（类型和日期）


`b.` **reviews** - 它是之前协同过滤使用的主要DataFrame，其中包含用户和电影之间的所有互动。


`c.` **all_recs** - 一个字典，每个键都是一个用户，值是基于协同过滤推荐的电影列表

对于 **all_recs** 中使用协同过滤收到十部推荐电影的用户，我们不需要处理他们。但是，数据集中还有很多用户没有获得任何电影推荐。

-----

`1.` 首先，我们将查找数据集中没有收到本因为协同过滤而能够获得的 10 部推荐电影的所有用户。

In [2]:
users_with_all_recs = [] # Store user ids who have all their recommendations in this (10 or more)
users_who_need_recs = # Store users who still need recommendations here


SyntaxError: invalid syntax (<ipython-input-2-13ec522bacde>, line 2)

In [ ]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

### 基于内容的推荐机制

这一次，你将混合使用内容和协同过滤向用户推荐电影。这样能够在之前无法做出推荐的很多情形下获得电影推荐。     

`2.` 在查找推荐电影之前，先从高到低地对用户的评分进行排序。你将按照此顺序遍历 movies 并寻找其他相似的电影。

In [ ]:
# create a dataframe similar to reviews, but ranked by rating for each user


### 相似性

在协同过滤部分，你掌握了计算两个用户之间相似性（或距离）的多种方法。我们可以按照几乎相同的方法基于内容计算相似性。  

在很多情形下，计算项目之间相似性的最快速方法是使用矩阵乘法，前提是矩阵不像之前那样很稀疏。如果你不了解这种方法，请观看此 [3blue1brown 视频](https://www.youtube.com/watch?v=LyGKycYT2v0)，并且[这篇帖子](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity)简单介绍了此方法。

要提取从内容角度描述 DateFrame 中电影的矩阵，我们可以使用与**年份**和**类型**相关的指示变量。  

然后对此矩阵及它本身进行点积运算，获得电影之间相似性的矩阵注意，在以下矩阵乘法中，如果值 1 相互重叠，那么结果是 2，表明相似性很高。在第二个点积中，值 1 不重叠。所以点积结果是 0，表示相似性很低。

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

我们可以根据内容特征对电影矩阵进行点积运算，获得电影x电影矩阵，其中每个单元格表示两部电影之间的相似度。在下图中，可以看出，对于这部分数据来说，电影 1 与电影 8 最相似，电影 2 与电影 8 最相似，电影 3 与电影 9 最相似。矩阵对角线上的元素将表示电影与其本身的相似性，相似性肯定最高（并且将对应于原始电影内容矩阵中电影行里的数字 1）

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` 请创建一个 NumPy 数组，它是一个与电影年份（按年代划分）和电影类型相关的指示变量矩阵。请计算此矩阵与其本身（转置后）的点积，得出每部电影与所有其他电影的相似性矩阵。最终矩阵应该是 31245 x 31245。

In [ ]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns


# Take the dot product to obtain a movie x movie matrix of similarities


In [ ]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
assert dot_prod_movies[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

### 对于每位用户...


你已经有一个矩阵，其中每个用户的评分都已排序。还有另一个矩阵，其中每个轴表示电影，如果两部电影更相似，那么矩阵条目更大，如果不相似，则矩阵条目更小。这个矩阵衡量的是内容相似性。下面到了有趣的环节了。

对于每个用户，我们将执行以下操作：

i. 对于每部电影，查找最相似且用户没看过的电影。

ii.继续查找已评分电影，直到推荐 10 部电影或没有可推荐的其他电影。

最后提示下，你可能需要调整“最相似”条件，才能获得 10 部推荐电影。第一遍我仅使用了相互之间相似性最高的电影作为推荐电影。

`3.` 在以下单元格中，请完成以下每个函数，从而基于内容做出推荐。

In [ ]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    
    
    # pull the movie titles based on the indices
    
    
    return similar_movies
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    
    # How many users for progress bar
    
    
    # Create the progressbar

    
    # For each user
        
        # Update the progress bar


        # Pull only the reviews the user has seen

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user



    # finish the progress bar

    
    return recs

In [ ]:
recs = make_recs()

### 推荐效果如何？

做出推荐之后，向每个人提供一组推荐电影的效果如何？

`4.` 请通过以下单元格查看你能够向多少用户推荐电影，并看看无法向其推荐电影的用户有哪些特性。

In [ ]:
# Explore recommendations


In [ ]:
# Some characteristics of my content based recommendations
users_without_all_recs = #store user ids without recs
users_with_all_recs = # users with all recs here
no_recs = # users with no recommendations

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

In [ ]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(189)

In [ ]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")
# Just one of many questions you could continue exploring...

### 下一步？  

如果你依然坚持采用一开始的电影相似性条件（就像我一开始那样），那么有些用户依然无法获得 10 部推荐电影（还有一小部分用户根本没有任何推荐电影）。 

我们之前就提到了，推荐引擎不仅是一门**科学**，而且是一项**艺术**。还有很多问题值得探索：协同过滤和基于内容的推荐机制相比如何？如何结合用户输入与协同过滤和/或基于内容的推荐来改进任何推荐系统？如何真正地为每位用户提供推荐内容？

`5.` 在最后一步，对于我们到目前为止介绍过的推荐技巧，你可以随意探索你的最后想法。你可能会选择对排名靠前的电影应用第一个技巧，并作出最后的推荐。你还可能会降低判定电影之间相似性的条件。大胆创新吧，别忘了与其他同学分享你的创意哦！

In [ ]:
# Cells for exploring

```python

```